<a href="https://colab.research.google.com/github/BenjaminDKLuong/Colab_Notes/blob/master/Detect_Mean_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DECTECT MEAN TWEETS**

In [1]:
!git clone https://github.com/PacktPublishing/Real-World-Python-Deep-Learning-Projects.git

fatal: destination path 'Real-World-Python-Deep-Learning-Projects' already exists and is not an empty directory.


In [0]:
# change working directory to new location
import os
os.chdir("/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source")


In [2]:
# check current working directory
%pwd

'/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source'

In [3]:
!ls

conf.py		models		    Real-World-Python-Deep-Learning-Projects
data		poldata.README.2.0  stopwords.py
data.tar.gz	prep.py		    train.py
mean_or_not.py	__pycache__	    txt_sentoken


In [7]:
import shutil
try:
  shutil.rmtree('/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source/models')
except:
  print('fail to delete')
else:
  print('delete sucessfully')

fail to delete


## Download Data

In [5]:
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
	if not os.path.isfile(file):
		print("Download file... " + file + " ...")
		urlretrieve(url,file)
print("File downloaded")

download('http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz','data.tar.gz')
print("All the files are downloaded")



File downloaded
All the files are downloaded


In [0]:
import tarfile

def uncompress_features_labels(dir):

  if (dir.endswith("tar.gz")):
    tar = tarfile.open(dir, "r:gz")
    tar.extractall()
    tar.close()
  elif (dir.endswith("tar")):
    tar = tarfile.open(dir, "r:")
    tar.extractall()
    tar.close()
			
uncompress_features_labels('data.tar.gz')

In [7]:
'''import shutil
import os

source = '/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source/txt_sentoken/pos/'
dest1 = '/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source/data/txt_sentoken/pos/'

try:
  os.mkdir(dest1)
except:
  print('dir exist')
  
files = os.listdir(source)

for f in files:
        shutil.move(source+f, dest1)'''

"import shutil\nimport os\n\nsource = '/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source/txt_sentoken/pos/'\ndest1 = '/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source/data/txt_sentoken/pos/'\n\ntry:\n  os.mkdir(dest1)\nexcept:\n  print('dir exist')\n  \nfiles = os.listdir(source)\n\nfor f in files:\n        shutil.move(source+f, dest1)"

## Import Stopwords

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Process Data

In [9]:
from os import listdir
from os import path

from stopwords import stopwords as exclude

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from pprint import pprint

def gen_x(xtext, tokenizer, max_len=None, for_training=False, ):
    print("Let's tokenize!")
    # We "fit" our tokenizer on our training set.
    # This is where unique numbers are generated for each word.
    if for_training:
        tokenizer.fit_on_texts(xtext)

    # Encode words(tokens) as unique numbers.
    encoded_xtext = tokenizer.texts_to_sequences(xtext)

    # We're looking for the longest sentence
    # in our training set.
    # Then we will use it when we ran gen_x on test data.
    # The key here is to have maximum lenght all the same trougout
    # training and data sets.
    if not max_len:
        max_len = max([len(s.split()) for s in xtext])
        tokenizer._max_padding_len=max_len

    # We need to pad our encoded text to the maximum lenght
    # for our embedding layer to work properly.
    train_x = pad_sequences(encoded_xtext, maxlen=max_len, padding='post')

    if for_training:
        return train_x, max_len
    return train_x

def cleanup(w, clean_sw=True):
    """
    Return a word if it's significant
    and None if it can be filtered out.
    clean_sw - should we filter out stop words?
    """
    w=w.strip().lower()
    if not w.isalpha():
        return None
    if clean_sw and w in exclude:
        return None
    if len(w) == 1:
        return None
    return w

def clean(data, clean_sw):
    """
    Remove unnecessary words and characters
    from a data set.
    data - a list of sentences to clean
    clean_sw - should we filter out stop words?
    """
    out=[]
    for doc in data:
        wout=[]
        for w in doc.split():
            w=cleanup(w, clean_sw)
            if w == None:
                continue
            wout.append(w)
        out.append(' '.join(wout))
    return out

def get_data(d='txt_sentoken', do_cleanup=True, filter_stopwords=True):
    """
    Load all our data into memory,
    split into training and data sets,
    clean up and encode, so we can use it
    with our neural network.
    do_cleanup - should we remove insignificant characters and words?
    filter_stopwords - should we remove common words?
    """
    train_x=[]
    train_y=[]

    test_x=[]
    test_y=[]

    # First, load all of the data into train_x.
    print('Loading data...')
    for p in ['neg', 'pos']:
        for filename in listdir(path.join(d,p)):
            dfile = path.join(d,p,filename)
            data=open(dfile).read()
            train_x.append(data)

    if do_cleanup:
        print('Doing cleanup...')
        ct=clean(train_x, filter_stopwords)
    else:
        ct=train_x

    # Split our data set as training and test set.
    # We have 1000 positive and 1000 negative reviews.
    l=1000
    # We split our data into 90% of data for training set
    # and we leave 10% for testing.
    trainl=int(l*0.90)
    testl=int(l*0.10)

    # First, spliting training set.
    # Negative first.
    train_x_neg=ct[0:trainl]
    train_x_pos=ct[l:l+trainl]

    # Generate approriate labels for negative data.
    # 0 means negative, 1 positive.
    train_y_neg=[ 0 for i in range(len(train_x_neg))]
    train_y_pos=[ 1 for i in range(len(train_x_pos))]

    # Put all of training splits together.
    train_x=train_x_neg+train_x_pos
    train_y=train_y_neg+train_y_pos

    # Get the remining 10% of data as test set.
    test_x_neg=ct[trainl:l]
    test_x_pos=ct[l+trainl:]

    test_y_neg=[ 0 for i in range(len(test_x_neg))]
    test_y_pos=[ 1 for i in range(len(test_x_pos))]

    test_x=test_x_neg+test_x_pos
    test_y=test_y_neg+test_y_pos

    # Create a new tokenizer, we will use it for both
    # training and test data.
    tokenizer=Tokenizer()
    # Encode and pad our train and test data.
    input_train_x = train_x
    train_x, max_len = gen_x(train_x, tokenizer, for_training=True)
    test_x = gen_x(test_x, tokenizer, max_len=max_len)

    # Just show a sample of input text and encoded text.
    print('Output from tokenizer:')
    pprint(input_train_x[0][:50])
    pprint(train_x[0][:9])
    
    for w in input_train_x[0][:50].replace(':','').split():
        if w in tokenizer.word_index.keys():
            print(w, '=', tokenizer.word_index[w])
    print()


    # Get a vocabulary size (a number of unique words).
    # We will later have to use it for our Embedding layer.
    inputs = len(tokenizer.word_index) + 1
    print('Vocab size:')
    print(inputs)
    return train_x, train_y, test_x, test_y, inputs, max_len, tokenizer



train_x, train_y, test_x, test_y, inputs, max_len, t=get_data()
print('X[0]', train_x[0])
print('Y[0]', train_y[0])

Using TensorFlow backend.


Loading data...
Doing cleanup...
Let's tokenize!
Let's tokenize!
Output from tokenizer:
'guess wild bachelor party gone really bad would br'
array([ 470,  421, 6382,  614,  600,   21,   29,    9, 1537], dtype=int32)
guess = 470
wild = 421
bachelor = 6382
party = 614
gone = 600
really = 21
bad = 29
would = 9

Vocab size:
36202
X[0] [ 470  421 6382 ...    0    0    0]
Y[0] 0


In [10]:
print('len train_x', len(train_x))
print('len train_y', len(train_y))
print('len test_x', len(test_x))
print('len test_y', len(test_y))



len train_x 1800
len train_y 1800
len test_x 200
len test_y 200


In [11]:
max_len

1299

## Set Up Model

In [0]:
import conf

from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D

import math
import os
import sys
import pandas
import pickle

from prep import get_data

def get_text_cnn(inputs, max_length, dim=25):
    """
    inputs - vocabulary size, a number of unique words in
            our data set
    max_lenght - the maximum number of words in our data set
    dim - word embedding dimension, the lenght of word vector
          that will be produced by this layer
    """
    print('CNN: inputs: %d, word embeddings dimesions: %d, input_length: %d' % (inputs, dim, max_length))
    model = Sequential()
    model.add(Embedding(inputs, dim, input_length=max_length))
    # Extract feature maps/most common "phrases".
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu', padding='same'))
    # Pick up the "best ones", pooling=reducting.
    model.add(MaxPooling1D(pool_size=4))
    # Just put everything together into one vector.
    model.add(Flatten())
    # This is the standard output for classification.
    # It matches our two classes 0 and 1.
    model.add(Dense(1, activation='sigmoid'))
    return model

confs={'default': dict(model=get_text_cnn)}

def train_model(name, train_x, train_y, epochs, batches, inputs, max_lenght, test_x, test_y):
    """
    Compile and train model with choosen parameters.
    """
    mparams=confs[name]
    model=mparams['model']
    model=model(inputs, max_lenght)
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit model on training data, validate during training on test data.
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=epochs, batch_size=batches, verbose=2)
    return model, name, mparams




In [20]:
dirName = 'models'
try:
	os.mkdir(dirName)
except:
  print('can not create models folder')

can not create models folder


In [21]:

# Getting our command line parameters
name = 'default'
epochs = 15
batches= 32


print('Train/Test Data lenght', len(train_x), len(test_x))

model, name, mp = train_model(name, train_x, train_y, epochs, batches, inputs, max_len , test_x, test_y)

# Save model to use for classification later on
mname='models/model-%s-%d-%d' % (name, epochs, batches)
model.save(mname+'.h5')

with open(mname+'-tokenizer.pickle', 'wb') as ts:
    pickle.dump(t, ts)
title='%s (epochs=%d, batch_size=%d)' % (name, epochs, batches)

# Test our model on both data that has been seen
# (training data set) and unseen (test data set)
print('Evaluation for %s' % title)

loss, acc = model.evaluate(train_x, train_y, verbose=2)
print('Train Accuracy: %.2f%%' % (acc*100))
print()

loss, acc = model.evaluate(test_x, test_y, verbose=2)
print('Test Accuracy: %.2f%%' % (acc*100))

Train/Test Data lenght 1800 200
CNN: inputs: 36202, word embeddings dimesions: 25, input_length: 1299
Train on 1800 samples, validate on 200 samples
Epoch 1/15
 - 1s - loss: 0.6885 - acc: 0.5383 - val_loss: 0.6843 - val_acc: 0.5550
Epoch 2/15
 - 0s - loss: 0.6371 - acc: 0.6483 - val_loss: 0.6757 - val_acc: 0.5550
Epoch 3/15
 - 0s - loss: 0.4997 - acc: 0.8228 - val_loss: 0.6155 - val_acc: 0.6750
Epoch 4/15
 - 0s - loss: 0.2093 - acc: 0.9856 - val_loss: 0.4551 - val_acc: 0.8050
Epoch 5/15
 - 0s - loss: 0.0482 - acc: 1.0000 - val_loss: 0.3902 - val_acc: 0.8400
Epoch 6/15
 - 0s - loss: 0.0156 - acc: 1.0000 - val_loss: 0.3747 - val_acc: 0.8250
Epoch 7/15
 - 0s - loss: 0.0076 - acc: 1.0000 - val_loss: 0.3672 - val_acc: 0.8400
Epoch 8/15
 - 0s - loss: 0.0047 - acc: 1.0000 - val_loss: 0.3637 - val_acc: 0.8450
Epoch 9/15
 - 0s - loss: 0.0032 - acc: 1.0000 - val_loss: 0.3635 - val_acc: 0.8500
Epoch 10/15
 - 0s - loss: 0.0024 - acc: 1.0000 - val_loss: 0.3638 - val_acc: 0.8450
Epoch 11/15
 - 0s - 

## Predict

In [22]:
from keras.models import load_model

# looking the training above, I choose epochs based on loss and val_loss
name = 'default'
epochs = 15
batches= 32


mname='models/model-%s-%d-%d' % (name, epochs, batches)

model_file=mname+'.h5'
tokenizer_file=mname+'-tokenizer.pickle'

# Loading the model.
if os.path.exists(model_file):
    model=load_model(model_file)
    print('Model loaded!')
else:
    print("Can't find %s model, train it first using 'train.py %s %d %d'" % (mname, name, epochs, batches))

    
if os.path.exists(tokenizer_file):
    tokenizer=pickle.load(open( mname+'-tokenizer.pickle', "rb" ))
    print('Tokenizer loaded!')
else:
    print("Can't find tokenizer for %s model, train it first using 'train.py %s %d %d'" % (mname, name, epochs, batches))
# Get the tweet.

Model loaded!
Tokenizer loaded!


In [28]:
tweet_original = input('Enter a tweet:  ')

Enter a tweet:  I hate you


In [29]:
# Negative = 0
# Positive = 1

tweet = clean([tweet_original],True)
print(tweet)
print()

test_tweet = gen_x(tweet, t, max_len=max_len)
print(test_tweet)
print()

pred=model.predict_classes(test_tweet)
pred=pred[0][0]
print('Predict: ',pred)

prob=model.predict_proba(test_tweet)
prob=prob[0][0]
print('Probability:',prob)
print()

print('%s -%smean(negative) (%.2f%%)' % (tweet_original.rstrip(), (' ' if pred==0 else ' not '),(1-prob)*100))

['hate']

Let's tokenize!
[[899   0   0 ...   0   0   0]]

Predict:  1
Probability: 0.5148021

I hate you - not mean(negative) (48.52%)


## I need to fix the model.  It doesnt work as expected.  The reason maybe because of cleaning tweets.  You can try "I'm happy" and "I'm not happy" as inputs.  After cleaning the tweets, we get "im happy" for both cases.